## Warmup
As shown on the picture, goal here is to compute 2^38. Easy peasy.

In [2]:
# Directory where files are stored. Will be used each time we need to load a file
file_directory = os.path.join(os.getcwd(), 'files')

In [8]:
print(pow(2, 38))

274877906944


## Step 01
Letters are moved from 2 positions. I use the ```ord``` function to get the ascii code and can then navigate and adapt.

In [9]:
str = "g fmnc wms bgblr rpylqjyrc gr zw fylb. rfyrq ufyr amknsrcpq ypc dmp. bmgle gr gl zw fylb gq glcddgagclr " \
            "ylb rfyr'q ufw rfgq rcvr gq qm jmle. sqgle qrpgle.kyicrpylq() gq pcamkkclbcb. lmu ynnjw ml rfc spj."
url = 'map'
output = ''

for character in str:
    ascii_code = ord(character)
    if ord('a') <= ascii_code <= ord('z'):
        if (ascii_code + 2) > ord('z'):
            output += chr(ord('a') - 1 + (ascii_code + 2) - ord('z'))
        else:
            output += chr(ascii_code + 2)
    else:
        output += character
print(output)

i hope you didnt translate it by hand. thats what computers are for. doing it in by hand is inefficient and that's why this text is so long. using string.maketrans() is recommended. now apply on the url.


## Step 02
Elements of the riddle found in the sourcecode of the page have been copied to ```step02.txt``` file.  
Here we need to find *rare* characters. My approach was to find all letters, no matter the case. It worked but did not really respect the riddle idea. I should have watched the number of occurrences because if there were also letters (but a few), this regexp would have found them as well.

In [3]:
import os
import re
# Assuming the kernel is running in this project directory (otherwise, adapt the line below)
file_to_load = os.path.join(file_directory, 'step02.txt')
content = ''
if os.path.exists(file_to_load):
    with open(file_to_load, 'rt') as f:
        content = f.read()
matches = re.findall('([a-zA-Z]+)', content)
for i in matches:
    print(i, end="")

equality

## Step 03
Elements of the riddle found in the sourcecode of the page have been copied to ```step03.txt``` file.  
Here we need to find in the text all small letters which are surrounded by EXACTLY 3 uppercase letters (before and after and exactly 3, no more).  
*Note: this code does not work if there are special characters in the content. Better way: `[^A-Z]` instead of [a-z]*

In [23]:
import os
import re
# Assuming the kernel is running in this project directory (otherwise, adapt the line below)
file_to_load = os.path.join(file_directory, 'step03.txt')
content = ''
if os.path.exists(file_to_load):
    with open(file_to_load, 'rt') as f:
        content = f.read()
matches = re.findall('[a-z][A-Z]{3}([a-z])[A-Z]{3}[a-z]', content)
for a_match in matches:
    print(a_match, end="")


linkedlist

## Step 04
Using requests to go to a new url each time we find a new number in the response content. Once the regexp does not match anymore it means we have solved the riddle. Next.

In [5]:
import requests
import re

url = 'http://www.pythonchallenge.com/pc/def/linkedlist.php?nothing='
#addon_num = '12345'
addon_num = '8022'
found = False
counter = 0
while not found:
    r = requests.get(url + addon_num)
    if r.text.find('and the next nothing is') < 0:
        found = True
        print('No more next nothing, found this text instead: "{}" for addon_num={}'.format(r.text, addon_num))
    else:
        addon_num = "".join(re.findall('.*and the next nothing is ([0-9]+)', r.text))
    counter += 1
    if counter % 25 == 0:
        print("{}: {}".format(counter, r.text))


25: and the next nothing is 64994
50: and the next nothing is 75494
75: and the next nothing is 76841
100: and the next nothing is 87890
125: and the next nothing is 73950
150: and the next nothing is 31785
No more next nothing, found this text instead: "peak.html" for addon_num=66831


## Step 05
Looking at the page codesource, we can see that there is an element with banner.p as source. The content has been copied into ```step05.p```file.  
A quick search on the internet and it seems that `p` is the extension for files saved with pickle module (ohhh ok...peak hell, pickle...ok...).  
Loaded the file with pickle and saw a list of tuples with either a space character either a dash, always followed by a number.   
Tried to check if there was any chance it would mean: number of occurrences of spaces/dashes to draw on each line.  
It worked. Next !

In [4]:
import pickle
import os

file_to_load = os.path.join(file_directory, 'step05.p')
content = pickle.loads(open(file_to_load, "rb").read())
for lines in content:
    line = [character * nb_occ for character, nb_occ in lines]
    print("".join(line))


                                                                                               
              #####                                                                      ##### 
               ####                                                                       #### 
               ####                                                                       #### 
               ####                                                                       #### 
               ####                                                                       #### 
               ####                                                                       #### 
               ####                                                                       #### 
               ####                                                                       #### 
      ###      ####   ###         ###       #####   ###    #####   ###          ###       #### 
   ###   ##    #### #######     ##  ### 

# Step 06

In [16]:
a = ' there are '
b = '"stylesheet'
c = 'annel.jpg">'

d = 'there are'
e = 'styleshee'
f = 'nnel.jpg"'

g = 'ere a'
h = 'ylesh'
i = 'el.jp'

j = 're '
k = 'les'
l = 'l.j'

print(''.join(''.join(x) for x in zip(a, b, c)))
print(''.join(''.join(x) for x in zip(d, e, f)))
print(''.join(''.join(x) for x in zip(g, h)))
print(''.join(''.join(x) for x in zip(j, k)))

print(''.join(''.join(x) for x in zip('now', 'there', 'are', 'pairs')))

 "atsnhtneyerllee. sjahpregee" t>
tsnhtneyerllee. sjahpregee"
eyrlee sah
rlee s
ntapohraweei
